In [1]:
!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 9.7 MB/s 
     |████████████████████████████████| 204 kB 52.6 MB/s 
     |████████████████████████████████| 352 kB 50.8 MB/s 
     |████████████████████████████████| 69 kB 7.2 MB/s 
     |████████████████████████████████| 2.3 MB 47.6 MB/s 
     |████████████████████████████████| 209 kB 42.0 MB/s 
     |████████████████████████████████| 254 kB 65.5 MB/s 
     |████████████████████████████████| 3.6 MB 45.6 MB/s 
     |████████████████████████████████| 5.6 MB 40.6 MB/s 
     |████████████████████████████████| 90 kB 8.6 MB/s 
     |████████████████████████████████| 11.0 MB 41.6 MB/s 
     |████████████████████████████████| 1.4 MB 47.6 MB/s 
     |████████████████████████████████| 2.4 MB 48.4 MB/s 
     |████████████████████████████████| 2.5 MB 50.7 MB/s 
     |████████████████████████████████| 506 kB 57.5 MB/s 
     |█████████████████████████████

# 1. 데이터 전처리

In [ ]:
import zipfile

# 데이터 불러오기 및 압축 풀기
dir = '/content/data/groove-v1.0.0-midionly.zip'
zipfile.ZipFile(dir).extractall()

In [ ]:
import pandas as pd

# info.csv를 확인
df = pd.read_csv('/content/groove/info.csv')
df.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [ ]:
from magenta.scripts.convert_dir_to_note_sequences import convert_directory

# convert_directory 함수를 이용하여 midi 파일을 tfrecord 파일로 변환
root_dir = '/content/groove'
output_file = '/content/data/tfrecord'

convert_directory(root_dir, output_file, recursive=True)

# 2. 모델

In [3]:
from magenta.models.music_vae.configs import Config
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.models.music_vae.base_model import MusicVAE
from magenta.common import merge_hparams
from magenta.contrib.training import HParams
import tensorflow as tf



Config.__new__.__defaults__ = (None,) * len(Config._fields)
CONFIG_MAP = {}
HParams = HParams

CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=16 * 4,  # 4 bars w/ 16 steps per bar
            z_size=256,
            enc_rnn_size=[512],
            dec_rnn_size=[256, 256],
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    tfds_name='groove/4bar-midionly',
)

In [ ]:
CONFIG_MAP
# 'Config': ['model', 'hparams', 'note_sequence_augmenter', 'data_converter','train_examples_path', 'eval_examples_path', 'tfds_name']

{'groovae_4bar': Config(model=<magenta.models.music_vae.base_model.MusicVAE object at 0x7f8c4c8bff10>, hparams=HParams([('batch_size', 512), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('control_preprocessing_rnn_size', [256]), ('dec_rnn_size', [256, 256]), ('decay_rate', 0.9999), ('dropout_keep_prob', 0.3), ('enc_rnn_size', [512]), ('free_bits', 48), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 64), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 256)]), note_sequence_augmenter=None, data_converter=<magenta.models.music_vae.data.GrooveConverter object at 0x7f8c4c8bf690>, train_examples_path=None, eval_examples_path=None, tfds_name='groove/4bar-midionly')}

모든 모델은 지수 감쇠율 0.9999 및 배치 크기 512로 10^-3에서 10^-5로 어닐링된 학습률로 학습 되었습니다.



# 3. 학습

In [ ]:
import os
from magenta.models.music_vae import configs
from magenta.models.music_vae import data
import tensorflow.compat.v1 as tf
import tf_slim

In [ ]:
# 텐서보드 요약 로그 저장 함수

def _trial_summary(hparams, examples_path, output_dir):
  examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

  hparams_dict = hparams.values()

  # Create a markdown table from hparams.
  header = '| Key | Value |\n| :--- | :--- |\n'
  keys = sorted(hparams_dict.keys())
  lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
  hparams_table = header + '\n'.join(lines) + '\n'

  hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

  with tf.Session() as sess:
    writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
    writer.add_summary(examples_path_summary.eval())
    writer.add_summary(hparam_summary.eval())
    writer.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 # 데이터로 부터 input tensor를 가져오는 함수
 
def _get_input_tensors(dataset, config):

  batch_size = config.hparams.batch_size
  iterator = tf.data.make_one_shot_iterator(dataset)
  (input_sequence, output_sequence, control_sequence,
   sequence_length) = iterator.get_next()
  input_sequence.set_shape(
      [batch_size, None, config.data_converter.input_depth])
  output_sequence.set_shape(
      [batch_size, None, config.data_converter.output_depth])
  if not config.data_converter.control_depth:
    control_sequence = None
  else:
    control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])
  sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }

In [ ]:
# train 함수

def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
  """Train loop."""
  # checkpoint 저장 경로
  tf.gfile.MakeDirs(train_dir)
  is_chief = (task == 0)
  """
  if is_chief:
    _trial_summary(
        config.hparams, config.train_examples_path or config.tfds_name,
        train_dir)
  """
  with tf.Graph().as_default():
    with tf.device(tf.train.replica_device_setter(
        num_ps_tasks, merge_devices=True)):

      model = config.model
      model.build(config.hparams,
                  config.data_converter.output_depth,
                  is_training=True)
      
      # optimizer는 Adam을 사용
      optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

      hooks = []
      if num_sync_workers:
        optimizer = tf.train.SyncReplicasOptimizer(
            optimizer,
            num_sync_workers)
        hooks.append(optimizer.make_session_run_hook(is_chief))

      grads, var_list = list(zip(*optimizer.compute_gradients(model.loss))) # 손실 함수는 Cross Entropy Loss를 사용

      # global_norm을 수행 
      global_norm = tf.global_norm(grads)
      tf.summary.scalar('global_norm', global_norm)

      if config.hparams.clip_mode == 'value':
        g = config.hparams.grad_clip
        clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
      elif config.hparams.clip_mode == 'global_norm':
        clipped_grads = tf.cond(
            global_norm < config.hparams.grad_norm_clip_to_zero,
            lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                grads, config.hparams.grad_clip, use_norm=global_norm)[0],
            lambda: [tf.zeros(tf.shape(g)) for g in grads])
      else:
        raise ValueError(
            'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
      train_op = optimizer.apply_gradients(
          list(zip(clipped_grads, var_list)),
          global_step=model.global_step,
          name='train_step')

      logging_dict = {'global_step': model.global_step,
                      'loss': model.loss}

      hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
      if num_steps:
        hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

      scaffold = tf.train.Scaffold(
          saver=tf.train.Saver(
              max_to_keep=checkpoints_to_keep,
              keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
      tf_slim.training.train(
          train_op=train_op,
          logdir=train_dir,
          scaffold=scaffold,
          hooks=hooks,
          save_checkpoint_secs=60,
          master=master,
          is_chief=is_chief)

In [ ]:
def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,):
  
  train_dir = '/content/train'
  config = config_map['groovae_4bar']
  def dataset_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf_file_reader,
        is_training=True,
        cache_dataset=True)
  
  train(
    train_dir,
    config=config,
    dataset_fn=dataset_fn,
    keep_checkpoint_every_n_hours=1,
    num_steps=100,
    master='',
    num_sync_workers=0,
    num_ps_tasks=0,
    task=0)
 

run(CONFIG_MAP)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  initializer=tf.constant_initializer(0.0))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.

# 4. Midi 파일 생성

In [5]:
from magenta.models.music_vae.trained_model import TrainedModel
import note_seq

model = TrainedModel(
    config=CONFIG_MAP['groovae_4bar'],
    batch_size=1,
    checkpoint_dir_or_path='/content/train') 

# 4마디 샘플을 생성
generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], '/content/sample/4bar_sample.mid')